In [1]:
import pandas as pd
import numpy as np
from datetime import date
from numpy import char
pd.set_option('display.float_format', str)

## Data

In [2]:
l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX", "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT","L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()
    
l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

In [3]:
o_columnnames = ["O_ORDERKEY", "O_CUSTKEY", "O_ORDERSTATUS", "O_TOTALPRICE", "O_ORDERDATE", "O_ORDERPRIORITY", "O_CLERK", "O_SHIPPRIORITY", "O_COMMENT"]

for i in range(len(o_columnnames)):
    o_columnnames[i] = o_columnnames[i].lower()
    
o_data_types = {
    'o_orderkey': int,
    'o_custkey': int,
    'o_orderstatus': str,
    'o_totalprice': float,
    'o_orderpriority': str,
    'o_clerk': str,
    'o_shippriority': int,
    'o_comment': str
}

o_parse_dates = ['o_orderdate']

In [4]:
c_columnnames = ["C_CUSTKEY", "C_NAME", "C_ADDRESS", "C_NATIONKEY", "C_PHONE", "C_ACCTBAL", "C_MKTSEGMENT", "C_COMMENT"]

for i in range(len(c_columnnames)):
    c_columnnames[i] = c_columnnames[i].lower()
    
c_data_types = {
    'c_custkey': int,
    'c_name': str,
    'c_address': str,
    'c_nationkey': int,
    'c_phone': str,
    'c_acctbal': float,
    'c_mktsegment': str,
    'c_comment': str
}

c_parse_dates = []

In [5]:
n_columnnames = ["N_NATIONKEY", "N_NAME", "N_REGIONKEY", "N_COMMENT"]

for i in range(len(n_columnnames)):
    n_columnnames[i] = n_columnnames[i].lower()
    
n_data_types = {
    'n_nationkey': int,
    'n_name': str,
    'n_regionkey': int,
    'n_comment': str,
}

n_parse_dates = []

### Lineitem DataFrame

In [6]:
# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../../tpch-pgsql-master/data/load/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

### Orders DataFrame

In [7]:
# Don't set indexes, as we can't access them with Pandas selection!
orders = pd.read_table("../../tpch-pgsql-master/data/load/orders.tbl.csv", sep="|", names=o_columnnames, dtype=o_data_types, parse_dates=o_parse_dates)

### Customer DataFrame

In [8]:
# Don't set indexes, as we can't access them with Pandas selection!
customer = pd.read_table("../../tpch-pgsql-master/data/load/customer.tbl.csv", sep="|", names=c_columnnames, dtype=c_data_types, parse_dates=c_parse_dates)

### Nation DataFrame

In [9]:
# Don't set indexes, as we can't access them with Pandas selection!
nation = pd.read_table("../../tpch-pgsql-master/data/load/nation.tbl.csv", sep="|", names=n_columnnames, dtype=n_data_types, parse_dates=n_parse_dates)

## Q10

In [10]:
import time
start_time = time.time()

df_filter_1 = customer[['c_custkey', 'c_name', 'c_address', 'c_nationkey', 'c_phone', 'c_acctbal', 'c_mktsegment', 'c_comment']]
df_filter_2 = lineitem[lineitem.l_returnflag == 'R']
df_filter_2 = df_filter_2[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_filter_3 = orders[(orders.o_orderdate >= pd.Timestamp('1993-10-01 00:00:00')) & (orders.o_orderdate < pd.Timestamp('1994-01-01 00:00:00'))]
df_filter_3 = df_filter_3[['o_custkey', 'o_orderkey']]
df_merge_1 = df_filter_2.merge(df_filter_3, left_on="l_orderkey", right_on="o_orderkey")
df_merge_1 = df_merge_1[['o_custkey', 'l_extendedprice', 'l_discount']]
df_merge_2 = df_filter_1.merge(df_merge_1, left_on="c_custkey", right_on="o_custkey")
df_merge_2 = df_merge_2[['c_custkey', 'c_name', 'c_acctbal', 'c_address', 'c_phone', 'c_comment', 'c_nationkey', 'l_extendedprice', 'l_discount']]
df_filter_4 = nation[['n_name', 'n_nationkey']]
df_merge_3 = df_merge_2.merge(df_filter_4, left_on="c_nationkey", right_on="n_nationkey")
df_merge_3 = df_merge_3[['c_custkey', 'n_name', 'c_name', 'l_extendedprice', 'l_discount', 'c_acctbal', 'c_address', 'c_phone', 'c_comment']]
df_sort_1 = df_merge_3.sort_values(by=['c_custkey', 'n_name'], ascending=[True, True])
df_sort_1 = df_sort_1[['c_custkey', 'n_name', 'c_name', 'l_extendedprice', 'l_discount', 'c_acctbal', 'c_address', 'c_phone', 'c_comment']]
df_group_1 = df_sort_1.groupby(['c_custkey', 'n_name'])
df_group_1 = df_group_1.apply(lambda s: pd.Series({
    "c_name":  s["c_name"].unique()[0],
    "revenue": (s["l_extendedprice"] * ( 1 - s["l_discount"] )).sum(),
    "c_acctbal":  s["c_acctbal"].unique()[0],
    "c_address":  s["c_address"].unique()[0],
    "c_phone":  s["c_phone"].unique()[0],
    "c_comment":  s["c_comment"].unique()[0],
}))
df_group_1 = df_group_1[['c_name', 'revenue', 'c_acctbal', 'c_address', 'c_phone', 'c_comment']]
df_sort_2 = df_group_1.sort_values(by=['revenue'], ascending=[False])
df_sort_2 = df_sort_2[['c_name', 'revenue', 'c_acctbal', 'c_address', 'c_phone', 'c_comment']]
df_limit_1 = df_sort_2.rename_axis(['c_custkey', 'n_name']).reset_index()
df_limit_1 = df_limit_1[['c_custkey', 'c_name', 'revenue', 'c_acctbal', 'n_name', 'c_address', 'c_phone', 'c_comment']]
end_time = time.time()
print(df_limit_1.head(20))

print("--- %s seconds ---" % (end_time - start_time))

    c_custkey              c_name           revenue  c_acctbal  \
0       57040  Customer#000057040       734235.2455     632.87   
1      143347  Customer#000143347 721002.6947999999    2557.47   
2       60838  Customer#000060838       679127.3077    2454.77   
3      101998  Customer#000101998 637029.5666999999    3790.89   
4      125341  Customer#000125341        633508.086    4983.51   
5       25501  Customer#000025501       620269.7849    7725.04   
6      115831  Customer#000115831       596423.8672     5098.1   
7       84223  Customer#000084223       594998.0239     528.65   
8       54289  Customer#000054289       585603.3918    5583.02   
9       39922  Customer#000039922 584878.1133999999    7321.11   
10       6226  Customer#000006226 576783.7605999999    2230.09   
11        922  Customer#000000922 576767.5332999999    3869.25   
12     147946  Customer#000147946        576455.132    2030.13   
13     115640  Customer#000115640       569341.1933     6436.1   
14      73

In [19]:
# Hesam Pandas Query
start_time = time.time()
ord_filt = orders[(orders.o_orderdate >= "1993-10-01") & (orders.o_orderdate < "1994-01-01")]

cu_proj = customer[["c_custkey", "c_name", "c_acctbal", "c_address", "c_nationkey", "c_phone", "c_comment"]]
ord_cu_join = pd.merge(cu_proj, ord_filt, left_on = "c_custkey", right_on = "o_custkey", how = "inner")

na_proj = nation[["n_nationkey", "n_name"]]
ord_na_join = pd.merge(na_proj, ord_cu_join, left_on = "n_nationkey", right_on = "c_nationkey", how = "inner")
ord_na_join = ord_na_join[["o_orderkey", "c_custkey", "c_name", "c_acctbal", "c_phone", "n_name", "c_address", "c_comment"]]

li_filt = lineitem[(lineitem.l_returnflag == "R")]

li_ord_join = pd.merge(ord_na_join, li_filt, left_on = "o_orderkey", right_on = "l_orderkey", how = "inner")

li_ord_join["revenue"] = li_ord_join.l_extendedprice * (1 - li_ord_join.l_discount)

result = li_ord_join \
    .groupby(["c_custkey", "c_name", "c_acctbal", "c_phone", "n_name", "c_address", "c_comment"]) \
    .agg(revenue=("revenue", "sum"))
    
result = result.sort_values(by=['revenue'], ascending=[False])
    
end_time = time.time()
print(result.head(20))

print("--- %s seconds ---" % (end_time - start_time))

                                                                                                                                                                            revenue
c_custkey c_name             c_acctbal c_phone         n_name         c_address                                c_comment                                                           
57040     Customer#000057040 632.87    22-895-641-3466 JAPAN          Eioyzjf4pp                               sits. slyly regular requests sleep alongside of...       734235.2455
143347    Customer#000143347 2557.47   14-742-935-3718 EGYPT          1aReFYv,Kw4                              ggle carefully enticing requests. final deposit... 721002.6947999999
60838     Customer#000060838 2454.77   12-913-494-9813 BRAZIL         64EaJ5vMAHWJlBOxJklpNc2RJiWE              need to boost against the slyly regular account         679127.3077
101998    Customer#000101998 3790.89   33-593-865-6378 UNITED KINGDOM 01c9CILnNtfOQYmZj             

In [18]:
# New Callum Query

start_time = time.time()

df_filter_1 = customer[['c_custkey', 'c_name', 'c_address', 'c_nationkey', 'c_phone', 'c_acctbal', 'c_mktsegment', 'c_comment']]
df_filter_2 = lineitem[lineitem.l_returnflag == 'R']
df_filter_2 = df_filter_2[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_filter_3 = orders[(orders.o_orderdate >= pd.Timestamp('1993-10-01 00:00:00')) & (orders.o_orderdate < pd.Timestamp('1994-01-01 00:00:00'))]
df_filter_3 = df_filter_3[['o_custkey', 'o_orderkey']]
df_merge_1 = df_filter_2.merge(df_filter_3, left_on="l_orderkey", right_on="o_orderkey")
df_merge_1 = df_merge_1[['o_custkey', 'l_extendedprice', 'l_discount']]
df_merge_2 = df_filter_1.merge(df_merge_1, left_on="c_custkey", right_on="o_custkey")
df_merge_2 = df_merge_2[['c_custkey', 'c_name', 'c_acctbal', 'c_address', 'c_phone', 'c_comment', 'c_nationkey', 'l_extendedprice', 'l_discount']]
df_filter_4 = nation[['n_name', 'n_nationkey']]
df_merge_3 = df_merge_2.merge(df_filter_4, left_on="c_nationkey", right_on="n_nationkey")
df_merge_3 = df_merge_3[['c_custkey', 'n_name', 'c_name', 'l_extendedprice', 'l_discount', 'c_acctbal', 'c_address', 'c_phone', 'c_comment']]
df_sort_1 = df_merge_3.sort_values(by=['c_custkey', 'n_name'], ascending=[True, True])
df_sort_1 = df_sort_1[['c_custkey', 'n_name', 'c_name', 'l_extendedprice', 'l_discount', 'c_acctbal', 'c_address', 'c_phone', 'c_comment']]
df_sort_1['revenue'] = df_sort_1.l_extendedprice * ( 1 - df_sort_1.l_discount )
df_group_1 = df_sort_1 \
    .groupby(['c_custkey', 'n_name', 'c_name', 'c_acctbal', 'c_address', 'c_phone', 'c_comment']) \
    .agg(
        revenue=("revenue", "sum"),
    )
df_group_1 = df_group_1[['revenue']]
df_sort_2 = df_group_1.sort_values(by=['revenue'], ascending=[False])
df_sort_2 = df_sort_2[['revenue']]
df_limit_1 = df_sort_2.rename_axis(['c_custkey', 'n_name', 'c_name', 'c_acctbal', 'c_address', 'c_phone', 'c_comment']).reset_index()
df_limit_1 = df_limit_1[['c_custkey', 'c_name', 'revenue', 'c_acctbal', 'n_name', 'c_address', 'c_phone', 'c_comment']]
end_time = time.time()
print(df_limit_1.head(20))

print("--- %s seconds ---" % (end_time - start_time))

    c_custkey              c_name           revenue  c_acctbal  \
0       57040  Customer#000057040       734235.2455     632.87   
1      143347  Customer#000143347 721002.6947999999    2557.47   
2       60838  Customer#000060838       679127.3077    2454.77   
3      101998  Customer#000101998       637029.5667    3790.89   
4      125341  Customer#000125341        633508.086    4983.51   
5       25501  Customer#000025501       620269.7849    7725.04   
6      115831  Customer#000115831       596423.8672     5098.1   
7       84223  Customer#000084223       594998.0239     528.65   
8       54289  Customer#000054289       585603.3918    5583.02   
9       39922  Customer#000039922       584878.1134    7321.11   
10       6226  Customer#000006226       576783.7606    2230.09   
11        922  Customer#000000922       576767.5333    3869.25   
12     147946  Customer#000147946        576455.132    2030.13   
13     115640  Customer#000115640       569341.1933     6436.1   
14      73

### Success!

Running the command in PSQL gives the output:

| c_custkey | c_name | revenue | c_acctbal | n_name | c_address | c_phone | c_comment |
|-----------|--------|---------|-----------|--------|-----------|---------|-----------|
| 57040 | Customer#000057040 | 734235.2455 |    632.87 | JAPAN                     | Eioyzjf4pp                               | 22-895-641-3466 | sits. slyly regular requests sleep alongside of the regular inst |
| 143347 | Customer#000143347 | 721002.6948 |   2557.47 | EGYPT                     | 1aReFYv,Kw4                              | 14-742-935-3718 | ggle carefully enticing requests. final deposits use bold, bold pinto beans. ironic, idle re |
| 60838 | Customer#000060838 | 679127.3077 |   2454.77 | BRAZIL                    | 64EaJ5vMAHWJlBOxJklpNc2RJiWE             | 12-913-494-9813 |  need to boost against the slyly regular account |
| 101998 | Customer#000101998 | 637029.5667 |   3790.89 | UNITED KINGDOM            | 01c9CILnNtfOQYmZj                        | 33-593-865-6378 | ress foxes wake slyly after the bold excuses. ironic platelets are furiously carefully bold theodolites |
| 125341 | Customer#000125341 | 633508.0860 |   4983.51 | GERMANY                   | S29ODD6bceU8QSuuEJznkNaK                 | 17-582-695-5962 | arefully even depths. blithely even excuses sleep furiously. foxes use except the dependencies. ca |
| 25501 | Customer#000025501 | 620269.7849 |   7725.04 | ETHIOPIA                  |   W556MXuoiaYCCZamJI,Rn0B4ACUGdkQ8DZ     | 15-874-808-6793 | he pending instructions wake carefully at the pinto beans. regular, final instructions along the slyly fina |
| 115831 | Customer#000115831 | 596423.8672 |   5098.10 | FRANCE                    | rFeBbEEyk dl ne7zV5fDrmiq1oK09wV7pxqCgIc | 16-715-386-3788 | l somas sleep. furiously final deposits wake blithely regular pinto b |
| 84223 | Customer#000084223 | 594998.0239 |    528.65 | UNITED KINGDOM            | nAVZCs6BaWap rrM27N 2qBnzc5WBauxbA       | 33-442-824-8191 |  slyly final deposits haggle regular, pending dependencies. pending escapades wake |
| 54289 | Customer#000054289 | 585603.3918 |   5583.02 | IRAN                      | vXCxoCsU0Bad5JQI ,oobkZ                  | 20-834-292-4707 | ely special foxes are quickly finally ironic p |
| 39922 | Customer#000039922 | 584878.1134 |   7321.11 | GERMANY                   | Zgy4s50l2GKN4pLDPBU8m342gIw6R            | 17-147-757-8036 | y final requests. furiously final foxes cajole blithely special platelets. f |
| 6226 | Customer#000006226 | 576783.7606 |   2230.09 | UNITED KINGDOM            | 8gPu8,NPGkfyQQ0hcIYUGPIBWc,ybP5g,        | 33-657-701-3391 | ending platelets along the express deposits cajole carefully final |
| 922 | Customer#000000922 | 576767.5333 |   3869.25 | GERMANY                   | Az9RFaut7NkPnc5zSD2PwHgVwr4jRzq          | 17-945-916-9648 | luffily fluffy deposits. packages c |
| 147946 | Customer#000147946 | 576455.1320 |   2030.13 | ALGERIA                   | iANyZHjqhyy7Ajah0pTrYyhJ                 | 10-886-956-3143 | ithely ironic deposits haggle blithely ironic requests. quickly regu |
| 115640 | Customer#000115640 | 569341.1933 |   6436.10 | ARGENTINA                 | Vtgfia9qI 7EpHgecU1X                     | 11-411-543-4901 | ost slyly along the patterns; pinto be |
| 73606 | Customer#000073606 | 568656.8578 |   1785.67 | JAPAN                     | xuR0Tro5yChDfOCrjkd2ol                   | 22-437-653-6966 | he furiously regular ideas. slowly |
| 110246 | Customer#000110246 | 566842.9815 |   7763.35 | VIETNAM                   | 7KzflgX MDOq7sOkI                        | 31-943-426-9837 | egular deposits serve blithely above the fl |
| 142549 | Customer#000142549 | 563537.2368 |   5085.99 | INDONESIA                 | ChqEoK43OysjdHbtKCp6dKqjNyvvi9           | 19-955-562-2398 | sleep pending courts. ironic deposits against the carefully unusual platelets cajole carefully express accounts. |
| 146149 | Customer#000146149 | 557254.9865 |   1791.55 | ROMANIA                   | s87fvzFQpU                               | 29-744-164-6487 |  of the slyly silent accounts. quickly final accounts across the |
| 52528 | Customer#000052528 | 556397.3509 |    551.79 | ARGENTINA                 | NFztyTOR10UOJ                            | 11-208-192-3205 |  deposits hinder. blithely pending asymptotes breach slyly regular re |
| 23431 | Customer#000023431 | 554269.5360 |   3381.86 | ROMANIA                   | HgiV0phqhaIa9aydNoIlb                    | 29-915-458-2654 | nusual, even instructions: furiously stealthy n |

This is similar to Pandas, so all good.

Time information (all times in seconds, to 3 s.f.):
| | Run 1 | Run 2 | Run 3 | Average |
| --- | --- | --- | --- | --- |
| Pandas | 45.9 | 37.4 | 37.3 | 40.2 |
| New Pandas | 1.90 | 1.89 | 1.99 | 1.93 |
| PostgreSQL | 2.43 | 1.94 | 1.65 | 2.01 |
| Hesam Pandas | 1.87 | 2.02 | 2.12 | 2.00 |



## Future Change Needed

My pandas creator is too slow, massively inefficient.
This is mostly likely due to the way we do group aggregation.
We use a _pd.Series_ applied (_apply_) to all the data.
This was chosen due to it being easier to implement, but is clearly now causing significant slowdown and performance impact.

We should instead move to using pandas's _groupby_ command.

First, we perform, on the previous DataFrame, any aggregations that we need to - save for the final part (sum/avg)
Then we group by all the keys save for the aggrs
Then we on the grouped dataframe can agg the aggrs with the final part of the command

We will benchmark between these two versions - but this is likely the cause of significant performance challenges

We will need to re-benchmark: Q1, Q3, Q10

| Query | Hesam | Old Version | New Version |
|---|---|---|---|
| Q1 | 2.29 | 4.03 | 3.47 |
| Q3 | 1.58 | 9.19 | 1.54 |
| Q10 | 2.00 | 40.2 | 1.93 |

**DONE: 03.11.2022**